In [ ]:
print(f"\n Hello Jupiter Colab \n")

In [7]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [28]:
file_paths = [
    "Data/Cat2600.zip",
    "Data/Dog2603.zip",
    "Data/CatAndDog.zip",
    # Add more file paths as needed
]

base_url = "https://github.com/Alvaro-MSJR/Training-Neural-Network-with-Transfer-Learning/tree/main/Data"

for file_path in file_paths:
    file_name = os.path.basename(file_path)  # Extract the file name
    url = base_url + file_path

    !echo "Downloading {file_name}"
    !curl -L -o sample_data/data/{file_name} -C - --progress-bar {url}



############################################################################################# 100.0%curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.

############################################################################################# 100.0%curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.

############################################################################################# 100.0%curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.



In [ ]:

# Juntar as partes do arquivo
!cat kagglecatsanddogs_5340.zip.001 kagglecatsanddogs_5340.zip.002 kagglecatsanddogs_5340.zip.003 > kagglecatsanddogs_5340.zip

# Extrair o arquivo zip
!unzip kagglecatsanddogs_5340.zip


Archive:  kagglecatsanddogs_5340.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of kagglecatsanddogs_5340.zip or
        kagglecatsanddogs_5340.zip.zip, and cannot find kagglecatsanddogs_5340.zip.ZIP, period.


In [27]:

# Extract the files if they are compressed (e.g., .zip, .tar.gz)
!unzip sample_data/data/Cat2600.zip

!tar -xf sample_data/data/Dog2603.zip

!tar -xzf sample_data/data/CatAndDog.zip

# Add extraction commands for other files as needed

Archive:  sample_data/Cat2600.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of sample_data/Cat2600.zip or
        sample_data/Cat2600.zip.zip, and cannot find sample_data/Cat2600.zip.ZIP, period.
tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors

gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [29]:
!ls ./sample_data/Data

ls: cannot access './sample_data/Data': No such file or directory


In [ ]:

# Remove the downloaded files if desired
!rm sample_data/Data/Cat2600.zip
!rm sample_data/Data/Dog2603.zip
!rm sample_data/Data/CatAndDog.zip

!ls

In [ ]:
!echo "Downloading kagglecatsanddogs_5340.zip.001 for image notebooks"
!curl -L -o kagglecatsanddogs_5340.zip.001 --progress-bar https://github.com/Alvaro-MSJR/Training-Neural-Network-with-Transfer-Learning/blob/main/Data/kagglecatsanddogs_5340.zip.001
!tar -xzf kagglecatsanddogs_5340.zip.001
!rm kagglecatsanddogs_5340.zip.001
!ls


##O=#  #                                                                                           

gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now
sample_data


In [ ]:
root = '101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

This function is useful for pre-processing the data into an image and input vector.

In [ ]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

Load all the images from root folder

In [ ]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

Randomize the data order.

In [ ]:
random.shuffle(data)

create training / validation / test split (70%, 15%, 15%)